In [ ]:
X_train = pd.read_csv('../datasets/word2vec/train_data_imputed_FINAL.csv')
X_test = pd.read_csv('../datasets/word2vec/test_data_imputed_FINAL.csv')
y_train = pd.read_csv('../datasets/word2vec/y_train_FINAL.csv').iloc[:,1:]
y_test = pd.read_csv('../datasets/word2vec/y_test_FINAL.csv').iloc[:,1:]
y_test = pd.read_csv('../datasets/word2vec/y_test_FINAL.csv').iloc[:,1:]

In [ ]:
X_train.head()

,location_AE,location_AM,location_AR,location_AT,location_AU,location_BD,location_BE,location_BG,location_BH,location_BR,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.415787,0.115367,-0.607734,-0.417199,0.090155,-0.399479,0.503071,0.088093,0.643417,0.180743
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.063942,-0.109937,-0.384977,-0.260925,-0.078151,-0.189497,0.363736,0.123283,0.343995,0.145450
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.309970,0.085213,-0.189873,-0.619696,0.238990,-0.288888,0.493239,0.069656,0.187424,0.184120
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.368265,-0.251463,-0.302825,0.120311,0.266548,-0.580960,0.567253,0.167470,0.057360,0.911378
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.103269,-0.057475,-0.195160,-0.509560,0.362500,0.138404,-0.018908,0.024695,0.141623,-0.178447


### Checking dataset

In [ ]:
X_train.shape

(11516, 263)

In [ ]:
X_test.shape

(2880, 263)

### Function to print metrics

In [ ]:
def print_metrics(y_true, y_pred, y_pred_prob):
    print("Confusion Matrix")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    
    print("F1 Score:", f1_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred)) 
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("ROC AUC:", roc_auc_score(y_true, y_pred_prob))

## Oversampling
- Using SMOTE-ENN



In [ ]:
original_counter = Counter(y_train.loc[:,'fraudulent'])
print("Before count", original_counter)

Before count Counter({0: 11293, 1: 223})


In [ ]:
def SMOTE_ENN(X_train, y_train, n_neighbours, k_neighbours, sampling_strategy):
    # SMOTE ENN oversampling
    smote_only = SMOTE(random_state=42, sampling_strategy=sampling_strategy, k_neighbors = k_neighbours)
    ennObj = EditedNearestNeighbours(n_neighbors=n_neighbours)
    smote_enn = SMOTEENN(random_state=42, smote=smote_only , enn= ennObj)
    X_res_smoteENN, y_res_smoteENN = smote_enn.fit_resample(X_train, y_train)
    return X_res_smoteENN, y_res_smoteENN

X_train_smenn, y_train_smenn = SMOTE_ENN(X_train, y_train, n_neighbours = 4, k_neighbours = 6, sampling_strategy = 0.1)

In [ ]:
after_counter = Counter(y_train_smenn.loc[:,'fraudulent'])
print("After count", after_counter)

After count Counter({0: 10968, 1: 1129})


## Hyperparameter Tuning using Random Search 
- Varying maximum iterations, solver, penalty and regularisation term

In [ ]:
random_grid = { 'max_iter': np.arange(100,500,100).tolist(),
                'solver' : ['liblinear', 'saga'],
                'penalty' : ['l2', 'l1'],
                'C': [0.001, 0.01, 0.1] 
                }

In [ ]:
lr_estimator = linear_model.LogisticRegression(verbose = 1)
lr_random = RandomizedSearchCV(estimator = lr_estimator, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=2022, n_jobs = -1, scoring='f1', refit=True)
lr_random.fit(X_train_smenn, y_train_smenn)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[LibLinear]

RandomizedSearchCV(cv=5, estimator=LogisticRegression(verbose=1), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'C': [0.001, 0.01, 0.1],
                                        'max_iter': [100, 200, 300, 400],
                                        'penalty': ['l2', 'l1'],
                                        'solver': ['liblinear', 'saga']},
                   random_state=2022, scoring='f1', verbose=2)

In [ ]:
lr_random.best_params_

{'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 100, 'C': 0.1}

In [ ]:
tuned_lr_estimator = lr_random.best_estimator_

In [ ]:
train_pred = tuned_lr_estimator.predict(X_train_smenn)
train_pred_proba = tuned_lr_estimator.predict(X_train_smenn)[:, 1]
print_metrics(y_train_smenn, train_pred, train_pred_proba)

Confusion Matrix
[[10868   100]
 [  373   756]]
F1 Score: 0.7617128463476072
Precision: 0.883177570093458
Recall: 0.6696191319751993
Accuracy: 0.9608993965445979
ROC AUC: 0.98348484907217


In [18]:
test_pred = tuned_lr_estimator.predict(X_test)
test_pred_proba = tuned_lr_estimator.predict_proba(X_test)[:, 1]
print_metrics(y_test, test_pred, test_pred_proba)

Confusion Matrix
[[2787   37]
 [  27   29]]
F1 Score: 0.47540983606557385
Precision: 0.4393939393939394
Recall: 0.5178571428571429
Accuracy: 0.9777777777777777
ROC AUC: 0.9585567583974098
